# Detection of social bots using tweets


#### Group 3: Stefani Platakidou, Kinga Jenei, Alejandro Lozada, Liam Le Tran



### Instalations

In [1]:
%pip install emoji

Note: you may need to restart the kernel to use updated packages.


### Imports

In [2]:
#Ignore print warnings
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

import nltk
from nltk.corpus import stopwords
import emoji

In [3]:
emoji.__version__

'2.2.0'

### Data presentation

In [4]:
df = pd.read_csv("data_sample_nlp.csv")
df.loc[20:23]

,label,id,author_id,created_at,type,text,language,geo_location,referenced_tweets_types,referenced_tweets_ids,user_mentions,media_types,in_reply_to_user_id,possibly_sensitive
20,bot,t1477711712191090689,2620097396,2022-01-02 18:41:41+00:00,post,@Quarry_Rock I had Harris as anytime scorer to...,en,False,{},{},{840817404},{},840817404.0,False
21,human,t1493923820893261831,1065878129963646976,2022-02-16 12:22:49+00:00,post,RT @forwardfooding: Alex Campos from @Nova_Mea...,en,False,{},{},"{2486723058,1065878129963646976}",{},NaN,False
22,human,t1499799906508173314,32817728,2022-03-04 17:32:17+00:00,post,I go into this all in more detail as part of m...,en,False,{},{},{91478624},{},32817728.0,False
23,bot,t1423386273784467463,37692343,2021-08-05 20:51:47+00:00,post,RT @EntheosAi: If we show an AI model millions...,en,False,{},{},{793303720205230080},{},NaN,False


In [5]:
df["text"][23]

'RT @EntheosAi: If we show an AI model millions of images and ask it to learn to categorize the world around us, what underlying geometry of…'

### Data pre-processing

In [6]:
# Download the stopwords collection from library
nltk.download('stopwords')
# Put it into a set to guarantee each word only appear once
STOPWORDS = list(set(stopwords.words('english')))
# Add punctuation to the stopwords list
STOPWORDS+=[".", "!", "?", ",", ";", ":", "[", "]", "{", "}", "-", "+", 
    "_", "/", "@", "#", "$", "%", "^", "&", "*", "(", ")", "<", ">", "|", "=",
    ".-", ".,", "'", '"', ',"', ".>", ".<"]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Liam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
"""
Questions : 
1. In general, what are the characteristics of a text written by a social bot?
2. Do they not contain tagged accounts? 
3. How do we handle emojis? Do they indicate that an account is a bot?
4. How do we handle other languages?

Notes: 
The number of unique words are still too high?
Very slow when remove emojies

"""

def preprocess(df):
    """"""
    #remove all languages except english
  
    
    # update tweets by lowercase, strip and tokenize
    unique_word_freqs = set()
    df['text'].str.lower().str.split().apply(unique_word_freqs.update)

    # exclude stop words, tagged accounts, punctuation, links and numbers    
    unique_word_freqs = list(unique_word_freqs)
    data = [word for i,word in enumerate(unique_word_freqs) if ( '@' not in word) and ('http' not in word) \
            and (word not in STOPWORDS) and (not word.isnumeric())]
    print(data[:100])
    
    # remove emojies
    clean_text = " ".join([word for word in data if not any(i in word for i in emoji.EMOJI_DATA)]).split()
    
    print(f'\nTotal unique words in all tweets: {len(clean_text)}\n')   
    return clean_text
    
data = preprocess(df)
print(data[:100])

['(needs', 'self-determined', 'l’intensification', '💪🏼🐠🌊', 'fuerte,', 'maphitha', 'ناراحته', 'condition…', 'والإعجاب', 'spacex’s', 'sospeitosos', '🤔¿qué', 'website👇', 'confiscation', '📌empatía,', 'ruhija', '(pre-fetch,', '(feels', 'destinaremos', 'mwst', 'sounds', '🗣massive', 'podiatry', '#chemclass', 'جرات', '#futureofpinkisgreen!', 'कॉन्फ्रेन्स', 'roste…', '#oneusinbeverlyhills', '(48.96%)', 'いよいよ開幕!手拍子に想いを乗せて。。。', 'رفتنشون', 'عليا', '#usacurling', '"those', 'coresident', 'marr', 'bruce?', '#stargatenow', 'haha,', '𝑖𝑠…', 'llms', 'continent?', 'luna....', 'marash', 'say),', 'reference”', '#solidarité', 'dices,', 'erru…', 'analizar', 'विद्यापीठात', 'smultr,age', 'juta', 'caroline,', '#cxp', 'ls4.', 'وغاز', '#rcotdatastratgy', 'relembrar', 'resources;', 'بمجرد', '#icassp2022', '#cozy', '#pharma', 'bus]', 'universal’s', 'ゴールとの距離、プレーの方向は要件を満たしてるとは思えないんですけどね。', 'away….', 'spiritual,', 'mcconnell’s', 'ai/pixel', 'میزنن!!!', 'crates', 'escenas,', 'nordics', 'kans?', '#yourlocalguil…', 'daydr

### Rep zng

### Build classifier

In [ ]:
def classifier():
    return 0

### Validation